In [1]:
import torch
from torch import optim, cuda
from torchvision import transforms, datasets, models
from pathlib import Path
import numpy as np
from timeit import default_timer as timer
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#%pip install torchsummary 
from torchsummary import summary

In [9]:
train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False

BASE_PATH = Path('/project/volume/data/out/NIMH-CHEFS')

TRAIN = 'train'
VAL = 'val'
TEST = 'test'

Train on gpu: False


In [10]:
# set transformations per dataset
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0)),
        transforms.ToTensor(), # automatically scales the from [0,255] to [0,1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], # vgg16 / imagenet standard
                             std=[0.229, 0.224, 0.225])

    ]),
    'val': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]) 
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]) 
    ])
}


# create datasets
image_datasets = {
    x: datasets.ImageFolder(
        root=(BASE_PATH / x),
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}


# create dataloaders to avoidd loading all fof the data into memory at once
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], 
        batch_size=64,
        shuffle=True, 
        num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = { x : len(image_datasets[x]) for x in [TRAIN, VAL, TEST] }

for x in [TRAIN, VAL, TEST]:
    print(f"[INFO] Number of images in {x} set ...{dataset_sizes[x]}")

class_names = image_datasets[TRAIN].classes
n_classes = len(class_names)
print(n_classes)
print("[INFO] Classes: ", image_datasets[TRAIN].classes)

[INFO] Number of images in train set ...320
[INFO] Number of images in val set ...106
[INFO] Number of images in test set ...107
5
[INFO] Classes:  ['Afraid', 'Angry', 'Happy', 'Neutral', 'Sad']


In [1]:
import torch
from torchvision import datasets, transforms

# Define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
}

# Create datasets
mnist_datasets = {x: datasets.MNIST(root='./data', train=(x=='train'), download=True, transform=data_transforms[x]) for x in ['train', 'val', 'test']}

# Create dataloaders
dataloaders = {x: torch.utils.data.DataLoader(mnist_datasets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(mnist_datasets[x]) for x in ['train', 'val', 'test']}

for x in ['train', 'val', 'test']:
    print(f"[INFO] Number of images in {x} set: {dataset_sizes[x]}")

class_names = [str(i) for i in range(10)]  # MNIST classes are numbers 0-9
n_classes = len(class_names)
print(n_classes)
print("[INFO] Classes: ", class_names)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████| 9912422/9912422 [00:11<00:00, 870568.59it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 264306.63it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████| 1648877/1648877 [00:02<00:00, 812503.75it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1717192.06it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

[INFO] Number of images in train set: 60000
[INFO] Number of images in val set: 10000
[INFO] Number of images in test set: 10000
10
[INFO] Classes:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [11]:
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
print(features.shape) # batch_size, color_channels, height, width
print(labels.shape) # batch_size

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [5]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

for param in vgg16.features.parameters():
    param.requires_grad = False

n_inputs = list(vgg16.classifier)[6].in_features

features = list(vgg16.classifier)[:-1]
features.extend([nn.Linear(in_features=n_inputs, out_features=n_classes)])

vgg16.classifier = nn.Sequential(*features)

total_params = sum(p.numel() for p in vgg16.parameters())
print(f'{total_params:,} total parameters.')

total_trainable_params = sum(p.numel() for p in vgg16.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

if train_on_gpu:
    vgg16 = vgg16.to('cuda')

summary(vgg16, input_size=(3, 224, 224), batch_size=64, device='cuda')

134,281,029 total parameters.
119,566,341 training parameters.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 224, 224]           1,792
              ReLU-2         [64, 64, 224, 224]               0
            Conv2d-3         [64, 64, 224, 224]          36,928
              ReLU-4         [64, 64, 224, 224]               0
         MaxPool2d-5         [64, 64, 112, 112]               0
            Conv2d-6        [64, 128, 112, 112]          73,856
              ReLU-7        [64, 128, 112, 112]               0
            Conv2d-8        [64, 128, 112, 112]         147,584
              ReLU-9        [64, 128, 112, 112]               0
        MaxPool2d-10          [64, 128, 56, 56]               0
           Conv2d-11          [64, 256, 56, 56]         295,168
             ReLU-12          [64, 256, 56, 56]               0
           Conv2d-13          [64, 256, 

In [12]:
class_to_idx = image_datasets[TRAIN].class_to_idx
idx_to_class = { idx: class_ for class_, idx in class_to_idx.items() }

## Training Loss and Optimizer

In [13]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(vgg16.parameters())

for p in optimizer.param_groups[0]['params']:
    if p.requires_grad:
        print(p)

Parameter containing:
tensor([[-0.0011, -0.0027,  0.0022,  ...,  0.0066, -0.0004, -0.0021],
        [ 0.0052,  0.0020,  0.0046,  ..., -0.0054, -0.0045, -0.0019],
        [-0.0005,  0.0052,  0.0018,  ...,  0.0068,  0.0005,  0.0091],
        ...,
        [-0.0075, -0.0096, -0.0025,  ..., -0.0079, -0.0106, -0.0036],
        [-0.0004,  0.0014, -0.0019,  ...,  0.0036,  0.0021,  0.0038],
        [ 0.0063,  0.0041, -0.0004,  ..., -0.0030,  0.0011,  0.0047]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0341,  0.0021,  0.0217,  ..., -0.0060,  0.0480,  0.0001],
       requires_grad=True)
Parameter containing:
tensor([[-0.0113,  0.0104, -0.0017,  ..., -0.0161,  0.0121,  0.0065],
        [-0.0005, -0.0078,  0.0071,  ..., -0.0041,  0.0099, -0.0111],
        [-0.0109, -0.0052,  0.0168,  ..., -0.0036,  0.0035, -0.0224],
        ...,
        [-0.0107, -0.0073, -0.0038,  ..., -0.0025,  0.0083, -0.0054],
        [ 0.0054,  0.0081, -0.0136,  ...,  0.0041, -0.0004, -0.0023],
        [ 0.02

In [14]:
def train(model, criterion, optimizer, train_loader, valid_loader, save_file_name, max_epoch_stop=3, n_epochs=20, print_every=1):

    # early stopping init
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting training from scratch.\n')

    overall_start = timer()

    for epoch in range(n_epochs):

        train_loss = 0.
        valid_loss = 0.

        train_acc = 0
        valid_acc = 0


        model.train()
        start = timer()

        for i, (data, target) in enumerate(train_loader):
            
            # move to gpu if possible
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            
            # zero out gradients after each step
            optimizer.zero_grad()
            # compute inference
            output = model(data)

            # compute loss 
            loss = criterion(output, target)
            # compute gradient according to loss
            loss.backward()

            # take step into direction of gradient (adjust weights)
            optimizer.step()

            # accumulate overall loss (average loss * nbr examples in batch)
            train_loss += loss.item() * data.size(0)

            # finde max log probalbility
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item() * data.size(0)

            print(f'Epoch: {epoch}\t{100 * (i + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',end='\r')


        # Start validation
        else:
            model.epochs += 1

            with torch.no_grad():
                model.eval()

                # Validation loop
                for data, target in valid_loader:

                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss (average loss * nbr examples in batch)
                    loss = criterion(output, target)
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy (average acc * nbr examples in batch)
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses and accuracy
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # training and validation results
                if (epoch + 1) % print_every == 0:
                    print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
                    print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                else:
                    epochs_no_improve += 1

                    # early stopping
                    if epochs_no_improve >= max_epoch_stop:
                        print(f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
                        total_time = timer() - overall_start
                        print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.')

                        # Load the best state dict and attach optimizer
                        model.load_state_dict(torch.load(save_file_name))
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 
                                'valid_loss', 
                                'train_acc',
                                'valid_acc'
                                ]
                            )
                        return model, history

In [15]:
model, history = train(
    vgg16,
    criterion,
    optimizer,
    dataloaders[TRAIN],
    dataloaders[VAL],
    save_file_name='vgg16-transfer-4.pt',
    max_epoch_stop=5,
    n_epochs=30,
    print_every=1
)


Starting training from scratch.



: 

: 

: 

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_loss', 'valid_loss']:
    plt.plot(
        history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Negative Log Likelihood')
plt.title('Training and Validation Losses')